In [1]:
# ensure installed packages can be used in the current notebook within the context of the current Python kernel
import sys
!{sys.executable} -m pip install pandas
!{sys.executable} -m pip install lxml

In [2]:
import pandas as pd
import numpy as np
import re
import requests
from lxml import html
from IPython.display import display, display_pretty, Javascript, HTML

# Flow

- [Process county-level results](#New-York-Times-data)
- [Process county-level geographies](#Census-geographies)
- [Combine datasets](#Combine-data)
- [Export county-level results](#Export-data)
- [Visualize](#Visualize)

## New York Times data

In [3]:
# connect to data

election_url = "https://static01.nyt.com/elections-assets/2020/data/api/2020-11-03/national-map-page/national/president.json"

# make an http request for the page
election_request = requests.request(
    method='GET', 
    url=election_url,
    headers={ "Accept": "application/json" }
)

election_response = election_request.json()

election_data = election_response['data']['races']

In [4]:
## parse response into dataframe, and select and rename final columns
election_data_df = pd.DataFrame(election_data)[['state_name', 'counties']].rename(columns={"state_name": "t_state_name"})

# after https://stackoverflow.com/a/49962887
# unnest 'counties' column, turning object keys in dataframe columns and object values into rows, select certain keys from each array, and rename those keys (columns)
election_data_df = pd.DataFrame(
    [
        dict(y, t_state_name=i) for i, x in zip(
            election_data_df['t_state_name'],
            election_data_df['counties']
        ) for y in x
    ]
)[['fips', 'name', 'votes', 't_state_name', 'results']].rename(columns={"fips": "geoid", "votes": "total_votes", "name": "t_county_name"})

# after https://stackoverflow.com/a/38231651
## unravel dictionary (JSON object) into other columns, choose final columns, rename them, and cast their data types
election_data_df = pd.concat(
    [
        election_data_df.drop(['results'], axis=1), 
        election_data_df['results'].apply(pd.Series)
    ], 
    axis=1
)[['geoid', 't_county_name', 'total_votes', 't_state_name', 'trumpd', 'bidenj']].rename(columns={"trumpd": "votes_gop", "bidenj": "votes_dem"}).astype({'votes_gop': 'int64', 'votes_dem': 'int64'})

# create state FIPS codes from the 5-digit 'geoid'
election_data_df['state_fips'] = election_data_df['geoid'].str[:2]

print(election_data_df.shape, election_data_df.dtypes)
election_data_df.head(10)

(3159, 7) geoid            object
t_county_name    object
total_votes       int64
t_state_name     object
votes_gop         int64
votes_dem         int64
state_fips       object
dtype: object


,geoid,t_county_name,total_votes,t_state_name,votes_gop,votes_dem,state_fips
0,02901,ED 1,7360,Alaska,3511,3477,02
1,02910,ED 10,11256,Alaska,8081,2727,02
2,02911,ED 11,10686,Alaska,7096,3130,02
3,02912,ED 12,11280,Alaska,7893,2957,02
4,02913,ED 13,7765,Alaska,4652,2666,02
5,02914,ED 14,11534,Alaska,6714,4261,02
6,02915,ED 15,5593,Alaska,2671,2622,02
7,02916,ED 16,8170,Alaska,3516,4274,02
8,02917,ED 17,7271,Alaska,2810,4136,02
9,02918,ED 18,7759,Alaska,2760,4681,02


## Census geographies

In [5]:
# get state geography data

# read state file csv and create dataframe
census_states = pd.read_csv(
    'https://www2.census.gov/geo/docs/reference/state.txt',
    delimiter='|',
    header=0,
    usecols=[0,1,2],
    names=['state_fips', 'state_abbr', 'c_state_name'],
    dtype=str
)

# define a list of FIPS codes of the internally-autonomous entities the U.S. has some control over
suzerainty_fips = ['60','66','69','72','74','78']

# filter out these Suzerainty entities
census_states = census_states[~census_states.state_fips.isin(suzerainty_fips)]

print(census_states.shape, '\n', census_states.dtypes)
census_states.tail(7)

(51, 3) 
 state_fips      object
state_abbr      object
c_state_name    object
dtype: object


,state_fips,state_abbr,c_state_name
44,49,UT,Utah
45,50,VT,Vermont
46,51,VA,Virginia
47,53,WA,Washington
48,54,WV,West Virginia
49,55,WI,Wisconsin
50,56,WY,Wyoming


In [6]:
# get county geography data

# base_url where all the county gazetteer files live
gazetteer_url = "https://www2.census.gov/geo/docs/maps-data/data/gazetteer/2019_Gazetteer/"

# make an http request for the page
page = requests.request(
    method='GET', 
    url=gazetteer_url,
    headers={ "Accept": "application/json" }
)

# parse the page and return a DOM tree
tree = html.fromstring(page.content)

# use XPath to return a list of link texts ('a' elements within the 'table' element) from the DOM
gazatteer_files = tree.xpath('//td/a/text()')

# filter the list to return only county file names
county_files = [c for c in gazatteer_files if re.match(r'.*counties.*\.txt', c)]

county_files

['2019_gaz_counties_01.txt',
 '2019_gaz_counties_02.txt',
 '2019_gaz_counties_04.txt',
 '2019_gaz_counties_05.txt',
 '2019_gaz_counties_06.txt',
 '2019_gaz_counties_08.txt',
 '2019_gaz_counties_09.txt',
 '2019_gaz_counties_10.txt',
 '2019_gaz_counties_11.txt',
 '2019_gaz_counties_12.txt',
 '2019_gaz_counties_13.txt',
 '2019_gaz_counties_15.txt',
 '2019_gaz_counties_16.txt',
 '2019_gaz_counties_17.txt',
 '2019_gaz_counties_18.txt',
 '2019_gaz_counties_19.txt',
 '2019_gaz_counties_20.txt',
 '2019_gaz_counties_21.txt',
 '2019_gaz_counties_22.txt',
 '2019_gaz_counties_23.txt',
 '2019_gaz_counties_24.txt',
 '2019_gaz_counties_25.txt',
 '2019_gaz_counties_26.txt',
 '2019_gaz_counties_27.txt',
 '2019_gaz_counties_28.txt',
 '2019_gaz_counties_29.txt',
 '2019_gaz_counties_30.txt',
 '2019_gaz_counties_31.txt',
 '2019_gaz_counties_32.txt',
 '2019_gaz_counties_33.txt',
 '2019_gaz_counties_34.txt',
 '2019_gaz_counties_35.txt',
 '2019_gaz_counties_36.txt',
 '2019_gaz_counties_37.txt',
 '2019_gaz_cou

In [7]:
# read each county file csv
census_county_files = [
    pd.read_csv(
        gazetteer_url + county_file,
        delimiter='\t',
        lineterminator='\n',
        header=0,
        usecols=[1,3],
        names=['geoid', 'c_county_name'],
        dtype=str
    ) for county_file in county_files]

# combine into a dataframe
census_counties = pd.concat(census_county_files, ignore_index=True)

# create state FIPS codes from the 5-digit 'geoid'
census_counties['state_fips'] = census_counties['geoid'].str[:2]

print(census_counties.shape, '\n', census_counties.dtypes)
census_counties.tail(10)

(3220, 3) 
 geoid            object
c_county_name    object
state_fips       object
dtype: object


,geoid,c_county_name,state_fips
3210,72135,Toa Alta Municipio,72
3211,72137,Toa Baja Municipio,72
3212,72139,Trujillo Alto Municipio,72
3213,72141,Utuado Municipio,72
3214,72143,Vega Alta Municipio,72
3215,72145,Vega Baja Municipio,72
3216,72147,Vieques Municipio,72
3217,72149,Villalba Municipio,72
3218,72151,Yabucoa Municipio,72
3219,72153,Yauco Municipio,72


In [8]:
# merge state and county geography data
census_geographies = census_states.merge(
    census_counties, 
    on='state_fips',
    how="left"
)

# drop and rename duplicate keys
census_geographies = census_geographies.drop(['state_fips', 'state_abbr'], axis=1)

print(census_geographies.shape, '\n', census_geographies.dtypes)
census_geographies.tail(10)

(3142, 3) 
 c_state_name     object
geoid            object
c_county_name    object
dtype: object


,c_state_name,geoid,c_county_name
3132,Wyoming,56027,Niobrara County
3133,Wyoming,56029,Park County
3134,Wyoming,56031,Platte County
3135,Wyoming,56033,Sheridan County
3136,Wyoming,56035,Sublette County
3137,Wyoming,56037,Sweetwater County
3138,Wyoming,56039,Teton County
3139,Wyoming,56041,Uinta County
3140,Wyoming,56043,Washakie County
3141,Wyoming,56045,Weston County


## Combine data

In [9]:
# merge county election data with county geographies

## merge to county geography files on county FIPS code
election_results = election_data_df.merge(
    right=census_geographies,
    on="geoid",
    how="left"
)

## create authoritative state and county name columns provided by Census
election_results['state_name'] = np.where(
    pd.isnull(election_results['c_state_name']), 
    election_results['t_state_name'], 
    election_results['c_state_name']
)

election_results['county_name'] = np.where(
    pd.isnull(election_results['c_county_name']), 
    election_results['t_county_name'], 
    election_results['c_county_name']
)

## replace Alaska District names
election_results['county_name'].replace('(ED )','District ', inplace=True, regex=True)

election_results = election_results.drop(["t_county_name", "t_state_name", "c_county_name", "c_state_name"], axis=1)

## filter dataframe to exclude Washington, D.C.
other_results = election_results[election_results['state_fips'] != '11']

### filter dataframe to just Washington, D.C. results
dc_results = election_results[election_results['state_fips'] == '11']

## rollup Washington, D.C. ward-level results to the state level
### aggregate votes and pluck first names
dc_results_aggregates = dc_results.groupby('state_fips', as_index=False).agg(
    { 'geoid': 'min', 'state_name': 'min', 'county_name': 'min', 'total_votes': 'sum', 'votes_gop': 'sum', 'votes_dem': 'sum' }
)

## combine DC and other county results, rename columns to match column names historically used in this repo
county_level_final = other_results.append(
    dc_results_aggregates, sort=True
).reset_index(
    drop=True
).drop(
    ['state_fips'], axis=1
).rename(
    columns={'geoid': 'county_fips'}
).sort_values(
    by='county_fips'
).reindex(
    columns=['state_name','county_fips','county_name','votes_gop','votes_dem','total_votes','diff','per_gop','per_dem','per_point_diff']
)

## add other metrics, including percentage of total votes captured by the GOP candidate, percentage of total votes captured by the DEM candidate, total difference between the two, and the percentage point difference
county_level_final['per_gop'] = np.where(
    county_level_final['total_votes'] == 0, 
    0, 
    county_level_final['votes_gop'] / county_level_final['total_votes']
)

county_level_final['per_dem'] = np.where(
    county_level_final['total_votes'] == 0, 
    0, 
    county_level_final['votes_dem'] / county_level_final['total_votes']
)

county_level_final['diff'] = county_level_final['votes_gop'] - county_level_final['votes_dem']
county_level_final['per_point_diff'] = county_level_final['per_gop'] - county_level_final['per_dem']

print(county_level_final.shape, '\n', county_level_final.dtypes)
county_level_final.tail(15)



(3152, 10) 
 state_name         object
county_fips        object
county_name        object
votes_gop           int64
votes_dem           int64
total_votes         int64
diff                int64
per_gop           float64
per_dem           float64
per_point_diff    float64
dtype: object


,state_name,county_fips,county_name,votes_gop,votes_dem,total_votes,diff,per_gop,per_dem,per_point_diff
3136,Wyoming,56017,Hot Springs County,1999,482,2577,1517,0.775708,0.187039,0.588669
3137,Wyoming,56019,Johnson County,3881,897,4914,2984,0.789784,0.182540,0.607245
3138,Wyoming,56021,Laramie County,27891,15217,44982,12674,0.620048,0.338291,0.281757
3139,Wyoming,56023,Lincoln County,8643,1509,10468,7134,0.825659,0.144154,0.681506
3140,Wyoming,56025,Natrona County,25271,8530,35202,16741,0.717885,0.242316,0.475570
3141,Wyoming,56027,Niobrara County,1118,155,1308,963,0.854740,0.118502,0.736239
3142,Wyoming,56029,Park County,12813,3410,16717,9403,0.766465,0.203984,0.562481
3143,Wyoming,56031,Platte County,3898,890,4923,3008,0.791794,0.180784,0.611010
3144,Wyoming,56033,Sheridan County,11843,4043,16428,7800,0.720903,0.246104,0.474799
3145,Wyoming,56035,Sublette County,3957,882,4970,3075,0.796177,0.177465,0.618712


## Export data

In [10]:
county_level_final.to_csv(
    '2020_US_County_Level_Presidential_Results.csv',
    sep=',',
    index=False
)

## Visualize

In [11]:
%%javascript
// https://github.com/d3/d3/issues/1693#issuecomment-35556356
require.config({
    paths: {
        "d3": "http://d3js.org/d3.v4",
        "d3scalechoromatic": "http://d3js.org/d3-scale-chromatic.v1.min",
        "topojson": "http://d3js.org/topojson.v1.min",
        "d3legend":"https://cdnjs.cloudflare.com/ajax/libs/d3-legend/2.11.0/d3-legend.min"
  },
    shim: {
        "d3scalechoromatic": ["d3.global"],
        "d3-interpolate":["d3.global"]
  }
});

define("d3.global", ["d3"], function(_) {
  this.d3 = _;
});

<IPython.core.display.Javascript object>

In [12]:
display(HTML("""
<style>
  .tract {
    stroke: #777;
    stroke-width: 0.05px;
    pointer-events: all;
  }
  .tract:hover {
    stroke: orange;
    pointer-events: all;

  }
  .tract-border {
    fill: none;
    /*stroke: #777;*/
    stroke-width: 0.05px;
    pointer-events: none;
  }
  @media (max-width: 767px) {
    .label {
      transform: rotate(-45deg) translate(-10px, 25px);
    }
  }
  .tract-border-state {
    fill: none;
    stroke: #333;
    stroke-width: 0.5px;
    pointer-events: none;
  }
  .legend {
    font-family: sans-serif;
    font-size: 10pt;

  }
  .legendTitle {
    font-weight: bold;
    font-size:11pt;
  }
  .background {
    fill:#f5f5f5;
  }
  body {
    background: #f5f5f5;
    /*font-family: sans-serif;*/
  }
  .bar {
    font-family: sans-serif;
    font-size: 8pt;
  }
  #cityLegend {
    font-family: sans-serif;
    text-anchor: middle;
  }
  #details .background {
    fill: #f5f5f5;
    fill-opacity: 0.7;
  }
  svg {
    width:100%;
  }
</style>
<div class="row">
  <div class="col-md-1 col-xs-1"></div>
  <div class="col-md-10 col-xs-10">
    <h2>2020 Presidential General Election Results by County</h2>
    <h4>Click on the map to explore</h4>
  </div>
</div>
<div class="row">
  <div class="col-md-12 col-xs-12">
    <svg ></svg>
  </div>
</div>
<div class="row">
  <div class="col-md-1 col-xs-1"></div>
  <div class="col-md-10 col-xs-10">
    <p>
      By Tony McGovern
    </p>

  </div> <!-- col-md-10 -->
  <div class="col-md-1">
  </div>

</div> <!-- .row -->
"""))

In [13]:
%%javascript

require(["d3", "topojson"], function(d3, topojson) {
    var svg = d3.select("svg"),
    width = $(document).width() * 10 / 12,
    height = $(document).height() - 200,
    margin = {
        top: 20,
        bottom: width > 767 ? 20 : 100,
        right: 20,
        left: 0
    },
    centered,
    fmt = d3.format(" >5.2%"),
    errorCount = 0;

svg.attr("width", width)
    .attr("height", height);

function ready(error, us, data) {
    if (error) throw error;


    var dictCities = {};
    data.forEach(function(d) {
        //Parse the percentages
        d["per_gop"] = +(d["per_gop"].slice(0, -1).replace(",", "."));
        d["per_dem"] = +(d["per_dem"].slice(0, -1).replace(",", "."));
        d.result = d["per_dem"] - d["per_gop"];
        d.combined_fips = +d.combined_fips;
        dictCities[d.combined_fips] = d;
    });

    var color = d3.scaleSequential(d3.interpolateRdBu)
        .domain([-1, 1]);

    // Add background
    svg.append("rect")
        .attr("class", "background")
        .attr("width", width)
        .attr("height", height)
        // .on("click", clicked);
        // To allow the zoom back
        // svg.on("click", clicked);
    var zoom = d3.zoom()
        .scaleExtent([1, 15])
        .on("zoom", zoomed);

    svg.style("pointer-events", "all")
        .call(zoom);
    var g = svg.append("g");

    function zoomed() {
        console.log(d3.event.transform);
        g.attr("transform", d3.event.transform);
    };

    var projection = d3.geoAlbersUsa()
        .scale(1280)
        .translate([width / 2, height / 2]);

    var path = d3.geoPath()
        .projection(projection);

    g.selectAll("path")
        .data(topojson.feature(us, us.objects.counties).features)
        .enter().append("path")
        .attr("class", "tract")
        .on("click", clicked)
        .on("mouseover", updateDetails)
        .style("fill", function(d) {
            var city = dictCities[d.id];
            if (city)
                return color(city.result);
            else {
                errorCount++;
                console.log(d.id + " Not found" + " errors = " + errorCount);
                return color(0);
            }
        })
        .attr("d", path)
        .append("title")
        .text(function(d) {
            var city = dictCities[d.id],
                county,
                state;
                
            // var msg = d.id;
            if (city) {
                county = city.county_name;
                state = city.state_abbr;
                var msg = county + ', ' + state + " Difference: " + fmt(city.result);
            }
            return msg;
        });


    // g.append("g")
    //     .attr("class", "counties")
    //   .selectAll("path")
    //     .data(topojson.feature(us, us.objects.counties).features)
    //   .enter().append("path")
    //     .attr("class", "tract-border")
    //     .attr("d", path);

    g.append("path")
        .datum(topojson.mesh(us, us.objects.states, function(a, b) {
            return a !== b;
        }))
        .attr("class", "tract-border-state")
        .attr("d", path);


    // g.append("path")
    //     .datum(topojson.mesh(mapData, mapData.objects.depts, function(a, b) { return a !== b; }))
    //     .attr("class", "tract-border-state")
    //     .attr("d", pathState);

    // The details
    var wScale = d3.scaleLinear()
        .domain([-1, 1])
        .range([-width / 3, width / 3]);
    var details_layer = svg.append("g")
        .attr("id", "details")
        .attr("transform", "translate(" + (width / 2 - 100) + ", 30)");
    details_layer.append("rect")
        .attr("class", "background")
        .attr("transform", "translate(" + (-wScale.range()[1] + 100) + ", -20)")
        .attr("width", wScale.range()[1] * 2 + 70)
        .attr("rx", 5)
        .attr("ry", 5)
        .attr("height", 60);
    details_layer.append("text")
        .attr("id", "cityLegend")
        .text("Difference")
        .attr("transform", "translate(100, 0)");

    var detailsBars = details_layer.selectAll("bar")
        .data([0.4978, -0.5021])
        .enter()
        .append("g")
        .attr("class", "bar");
    detailsBars
        .append("rect")
        .attr("width", 0)
        .attr("height", width > 767 ? 20 : 10)
        .attr("x", 100)
        .attr("y", 10)
        .style("fill", color)
        .transition()
        .duration(500)
        .attr("x", function(d) {
            return d > 0 ? 100 : 100 - wScale(-d);
        })
        .attr("width", function(d) {
            return d > 0 ? wScale(d) : wScale(-d);
        });
    detailsBars.append("text")
        .text(function(d) {
            return (d > 0 ? "" : "Rep ") +
                fmt(d > 0 ? d : -d) +
                (d > 0 ? " Dem" : "");
        })
        .attr("dx", function(d) {
            return d > 0 ? 5 : -5;
        })
        .attr("dy", 24)
        .attr("x", 100)
        .style("text-anchor", function(d) {
            return d > 0 ? "start" : "end";
        })
        .transition()
        .duration(500)
        .attr("x", function(d) {
            return d > 0 ? 100 + wScale(d) : 100 - wScale(-d);
        });



    // The legend
    svg.append("g")
        .attr("class", "legend")
        .attr("transform",
            width > 767 ?
            "translate(" + (width - margin.right - 150) + ",100)" :
            "translate(" + (width / 2 - 100) + "," + (height - 120) + ")"
        );

    var legendLinear = d3.legendColor()
        // .shapeWidth(30)
        .cells(7)
        .orient(width > 767 ? "vertical" : "horizontal")
        .title("Diferencia")
        .labels([
            " 100.00% Dem",
            "  66.67%",
            "  33.33%",
            "   0.00%",
            "  33.33%",
            "  66.67%",
            " 100.00% Rep",
        ].reverse())
        .labelFormat(fmt)
        .ascending(true)
        .labelAlign("end")
        .scale(color);

    svg.select(".legend")
        .call(legendLinear);

    // When clicked, zoom in
    function clicked(d) {
        updateDetails(d);
        var x, y, k;

        // Compute centroid of the selected path
        if (d && centered !== d) {
            // if (d) {
            var centroid = path.centroid(d);
            x = centroid[0];
            y = centroid[1];
            // k = zoom.scaleExtent()[1];
            k = 10;
            centered = d;
        }
        else {
            x = width / 2;
            y = height / 2;
            k = 1;
            centered = null;
        }



        // Manually Zoom
        svg.transition()
            .duration(750)
            .call(zoom.transform, d3.zoomIdentity
                .translate(width / 2, height / 2)
                .scale(k)
                .translate(-x, -y));
    }

    function updateDetails(d) {

        var data = [0.4978, -0.5021],
            name = "Difference " + fmt(data[0] + data[1]),
            state,
            county,
            city;
            
        if (d) {
            city = dictCities[d.id];
            if (city) {
                county = city['county_name'];
                state = city['state_abbr'];
                data = [city["per_dem"], -city["per_gop"]];
                name = county + ', ' + state + " Difference: " + fmt(data[0] + data[1]);
            }
        }
        // console.log(data);
        // console.log(name);
        var detailsBars = details_layer
            .selectAll(".bar")
            .data(data);

        detailsBars.select("rect")
            .transition()
            .duration(500)
            .attr("x", function(d) {
                return d > 0 ? 100 : 100 - wScale(-d);
            })
            .attr("width", function(d) {
                return d > 0 ? wScale(d) : wScale(-d);
            })
            .style("fill", color);

        detailsBars.select("text")
            .text(function(d) {
                return (d > 0 ? "" : "Rep ") +
                    fmt(d > 0 ? d : -d) +
                    (d > 0 ? " Dem" : "")
            })
            .transition()
            .duration(500)
            .attr("x", function(d) {
                return d > 0 ? 100 + wScale(d) : 100 - wScale(-d);
            })


        details_layer.select("#cityLegend").text(name);

    }
}

d3.queue()
    .defer(d3.json, "https://raw.githubusercontent.com/john-guerra/US_Elections_Results/master/us.json")
    .defer(d3.csv, "https://raw.githubusercontent.com/tonmcg/US_County_Level_Election_Results_08-20/master/2020_US_County_Level_Presidential_Results.csv")
    .await(ready);
    
});

<IPython.core.display.Javascript object>